In [40]:
import itk
import os
import icon_registration
import icon_registration.pretrained_models
import icon_registration.itk_wrapper
import monai
import icon_registration.monai_wrapper as mw
import pandas as pd


import numpy as np

data_folder = "/playpen-raid2/lin.tian/data/BraTS-Reg/BraTSReg_Training_Data_v3"
case_list = os.listdir(data_folder)
case_list = list(filter(lambda x: "BraTS" in x.split("/")[-1], case_list))

mtres = []
losses = []
import tqdm
model = icon_registration.pretrained_models.brain_registration_model(
        pretrained=True
        )

#model.similarity = mw.FirstChannelInputs(monai.losses.GlobalMutualInformationLoss())
for case_path in tqdm.tqdm(case_list[-1:]):
    file_list = os.listdir(f"{data_folder}/{case_path}")
    source_list = list(filter(lambda x: ("_00_" in x) and ("t1.nii" in x), file_list))
    target_list = list(filter(lambda x: ("_01_" in x) and ("t1.nii" in x), file_list))
    image_A = itk.imread(f"{data_folder}/{case_path}/{source_list[0]}")
    image_B = itk.imread(f"{data_folder}/{case_path}/{target_list[0]}")
    source_landmarks = pd.read_csv(
        f"{data_folder}/{case_path}/{[f for f in file_list if '_00_' in f and 'landmarks' in f][0]}"
    ).values[:, 1:]
    target_landmarks = pd.read_csv(
        f"{data_folder}/{case_path}/{[f for f in file_list if '_01_' in f and 'landmarks' in f][0]}"
    ).values[:, 1:]


    image_A_processed = icon_registration.pretrained_models.brain_network_preprocess(
            image_A
        )

    image_B_processed = icon_registration.pretrained_models.brain_network_preprocess(
        image_B
    )


    phi_AB, phi_BA, loss = icon_registration.itk_wrapper.register_pair(
            model, image_A_processed, image_B_processed, finetune_steps=50, return_artifacts=True
        )

    warped_target_landmarks = np.array([list(phi_AB.TransformPoint(t)) for t in target_landmarks * 1.0])

    mtres.append(np.sqrt(np.sum((source_landmarks - warped_target_landmarks)**2, axis=1)).mean())



100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:44<00:00, 44.17s/it]


In [45]:
np.mean(mtres)

2.7316132167766076

In [46]:
np.mean(mtres)

2.7316132167766076

In [47]:
mtres

[2.7316132167766076]

In [48]:
validation: 3.1489 mm

SyntaxError: invalid syntax (2242379696.py, line 1)

In [49]:
mtres

[2.7316132167766076]

In [50]:
20 / 140

0.14285714285714285

In [51]:
loss

ICONLoss(all_loss=1.3396021127700806, inverse_consistency_loss=0.00992486160248518, similarity_loss=1.3247147798538208, transform_magnitude=0.00013648413005284965, flips=168.0)